<a href="https://colab.research.google.com/github/greyhound101/multilingial/blob/master/tpu_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [2]:
pip install transformers

     |████████████████████████████████| 1.8MB 6.9MB/s 
     |████████████████████████████████| 890kB 20.9MB/s 
     |████████████████████████████████| 2.9MB 26.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ac5e1b538434d31dca0e8220f61f01144ac5c9cb0356227f577cb97422be69d2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  29918      0 --:--:-- --:--:-- --:--:-- 29918
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
     |████████████████████████████████| 61kB 3.7MB/s 
Uninstalling torch-1.7.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12


In [ ]:
import gc
import os
import time
import math
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import date
from transformers import *
from sklearn.metrics import *
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F

import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
from torch import Tensor
from torch.optim import *
from torch.nn.modules.loss import *
from torch.optim.lr_scheduler import * 
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import RandomSampler
import pandas as pd
def load_data():
    trn=pd.read_csv('/content/gdrive/My Drive/multilingual/jigsaw-toxic-comment-train.csv.zip',usecols=['toxic','comment_text'])
    tst=pd.read_csv('/content/gdrive/My Drive/multilingual/test.csv.zip',usecols=['lang','content'])  
    sub=pd.read_csv('/content/gdrive/My Drive/multilingual/submission.csv')
    val=pd.read_csv( '/content/gdrive/My Drive/multilingual/validation.csv.zip')  
    return trn,tst,sub,val


def get_lang(val,tst,lang):
  df=pd.concat([val,tst],0)
  return df.loc[df['lang']==lang].reset_index(drop=True).drop(['id','lang'],1)


def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

class Transformer(nn.Module):
    def __init__(self, transformer, num_classes=1):
        """
        Constructor
        
        Arguments:
            model {string} -- Transformer to build the model on. Expects "camembert-base".
            num_classes {int} -- Number of classes (default: {1})
        """
        super().__init__()
        
        self.transformer = transformer

        self.nb_features = self.transformer.pooler.dense.out_features
        for param in self.transformer.parameters():
          param.requires_grad = False
        self.pooler = nn.Sequential(
            nn.Linear(self.nb_features, num_classes), 
            nn.Sigmoid(),
        )


    def forward(self, tokens):
        """
        Usual torch forward function
        
        Arguments:
            tokens {torch tensor} -- Sentence tokens
        
        Returns:
            torch tensor -- Class logits
        """
        hidden_states = self.transformer(
            tokens, attention_mask=(tokens > 0).long()
        )[1]

        # hidden_states = hidden_states[-1][:, 0] # Use the representation of the first token of the last layer

        ft = self.pooler(hidden_states)

        return ft


In [ ]:
class bce(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(bce, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        one=(1-targets)*torch.log(1-inputs)
        zero=(targets*torch.log(inputs))
        loss = torch.mean((one+zero)*-1)
        
        return loss
class JigsawDataset:
    """
    Torch dataset for training and validating
    """
    def __init__(self, x,y):
        super().__init__()
        self.y = y 
        self.sentences = x
        

    def __len__(self):
        return self.sentences.shape[0]

    def __getitem__(self, idx):
      len=self.__len__()
      if idx>len:
        idx=idx%len
      
      return torch.tensor(self.sentences[idx]), torch.tensor(self.y[idx]).float()



In [ ]:
from sklearn.model_selection import train_test_split
from statistics import mean
import torch_xla
from sklearn.preprocessing import *
import torch_xla.debug.metrics as met
import torch_xla.distributed.data_parallel as dp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
import gc
import os
import cv2
import zipfile
from pathlib import Path
import random
import argparse
import sys
from statistics import mean
import yaml
from tqdm.notebook import tqdm
import time
import albumentations as A
from sklearn.model_selection import StratifiedKFold
import random
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from typing import Dict
from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt

def train_all(train_loader, model, device, optimizer):
    model.train()
    # t = tqdm(train_loader, disable=not xm.is_master_ordinal())
    model.train()
    lss=bce()
    loss1=[]
    for step, (x, y_batch) in enumerate(train_loader): 
            
            # x = x.to(device)
            y_batch = y_batch.to(device)
            y_pred = model(x)
            
            loss = lss(y_pred.view(-1).float(), y_batch.float())
            loss.backward()
            loss1.append(loss.item())
            xm.optimizer_step(optimizer)
            
            model.zero_grad()
    return mean(loss1)

def valid_all(train_loader, model, device):
    # t = tqdm(train_loader, disable=not xm.is_master_ordinal())
    lss=bce()
    loss1=[]
    for step, (x, y_batch) in enumerate(train_loader): 
            
            # x = x.to(device)
            y_batch = y_batch.to(device)
            y_pred = model(x)
            
            loss = lss(y_pred.view(-1).float(), y_batch.float())
            loss.backward()
            loss1.append(loss.item())
            
    return mean(loss1)

def load_data(lang):
    with zipfile.ZipFile('/content/gdrive/My Drive/multilingual/jigsaw-toxic-comment-train-google-'+lang+'-cleaned.csv.zip', 'r') as zip_ref:
      zip_ref.extractall('')
    trn=pd.read_csv('/content/jigsaw-toxic-comment-train-google-'+lang+'-cleaned.csv',usecols=['toxic','comment_text'])
    trn['lang']=lang
    tst=pd.read_csv('/content/gdrive/My Drive/multilingual/test.csv.zip',usecols=['lang','content'])  
    sub=pd.read_csv('/content/gdrive/My Drive/multilingual/submission.csv')
    val=pd.read_csv( '/content/gdrive/My Drive/multilingual/validation.csv.zip',usecols=['lang','comment_text'])  
    tst.columns=['comment_text','lang']
    tst['toxic']=sub['toxic']
    df=pd.concat([trn,tst,val],0)
    return df.loc[df['lang']==lang].reset_index(drop=True).drop(['lang'],1)


def get_lang(val,tst,lang):
  df=pd.concat([val,tst],0)
  return df.loc[df['lang']==lang].reset_index(drop=True).drop(['id','lang'],1)

def main():
    epochs=1
    batch_size=32
    learning_rate=1e-3
    seed=42

    # Setting seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

    df=load_data('es')
    tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
    x_train = regular_encode(list(df.comment_text.values), tokenizer, maxlen=128)
    y_train = df.toxic.values
    X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=42)

    




    def run():

        torch.manual_seed(seed)

        device = xm.xla_device()
        model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
        model=Transformer(model).to(device)


        #Training
        train_dataset = JigsawDataset(X_train,Y_train)


        train_sampler = torch.utils.data.distributed.DistributedSampler(
            train_dataset,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=False
        )
        train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,
            sampler=train_sampler,
            drop_last=False,
            num_workers=2
        )

        

        #Validation
        validation_dataset = JigsawDataset(X_test,Y_test)


        validation_sampler = torch.utils.data.distributed.DistributedSampler(
            validation_dataset,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=False
        )
        validation_loader = torch.utils.data.DataLoader(validation_dataset,batch_size=batch_size,
            sampler=validation_sampler,
            drop_last=False,
            num_workers=2
        )

        num_train_steps = int(df.shape[0] / batch_size / xm.xrt_world_size() * epochs)

        optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate * xm.xrt_world_size(),weight_decay=1e-3)


        xm.master_print("Training is Starting ...... ")
        total_loss=[]
        valid_loss=[]
        for i in tqdm(range(50)):
          para_loader = pl.ParallelLoader(train_loader, [device])
          total_loss.append(train_all(para_loader.per_device_loader(device),model,device,optimizer))
          para_loader = pl.ParallelLoader(validation_loader, [device])
          valid_loss.append(valid_all(para_loader.per_device_loader(device),model,device))
        np.save('loss.npy',total_loss)
        np.save('valid.npy',valid_loss)
            


    def _mp_fn(rank, flags):
        torch.set_default_tensor_type('torch.FloatTensor')
        run()
        
    FLAGS={}
    xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

In [ ]:
main()

In [ ]:
import seaborn as sns
a=np.load('loss.npy',allow_pickle=False)
sns.scatterplot(y=a,x=range(len(a)))

In [ ]:

a=np.load('valid.npy',allow_pickle=False)
sns.scatterplot(y=a,x=range(len(a)))